In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf
import talib
from utils import utils
import os

In [58]:
df = pd.read_csv('../data/USDJPY.csv')
df.sample(5)

,Gmt time,Open,High,Low,Close,Volume
71580,13.10.2014 13:00:00.000,107.286,107.401,107.201,107.305,8520.7998
38478,25.06.2009 20:00:00.000,95.865,96.014,95.823,95.947,15952.0000
93084,27.03.2018 07:00:00.000,105.676,105.687,105.466,105.495,8117.9302
6351,09.05.2004 23:00:00.000,112.235,112.952,112.220,112.952,14878.5000
57570,13.07.2012 08:00:00.000,79.298,79.316,79.250,79.258,3287.0400


In [59]:
df = utils.rename_columns_and_format(df)
df = utils.get_all_indicators(df)

df.dropna(inplace=True)

## Funcion para obtener el modelo

In [130]:
def get_model(x_input, y_input):
    model = tf.keras.models.Sequential()
    
    #kernel = 2,1 o 1,2 y maxpooling tmb
    #tamano kernel mas grande (dias)
    model.add(tf.keras.layers.Conv2D(32, (x_input, 3), input_shape=(x_input, y_input,1), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((1, 2)))
    
    model.add(tf.keras.layers.Conv2D(32, (1, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((1, 2)))

    model.add(tf.keras.layers.Conv2D(32, (1, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((1, 2)))
    
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    #model.summary()
    
    return model

## Prueba utilizando indicadores individuales como inputs

In [138]:
WINDOW = 30
P_TRAIN = 0.8
EPOCHS = 100
BATCH_SIZE = 32
df = df.head(500)

columns = ['ema_12', 'ema_26', 'upper_bband', 
         'middle_bband', 'lower_bband', 'rsi', 
         'macd', 'macd_signal', 'macd_hist', 'k', 'd']

target_column = df['close'].to_numpy()
for column in columns:
    df_min = df[column]
    x, y = utils.create_windowed_dataset(df_min, target_column, WINDOW)
    x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)
    
    _, x_input, y_input = x.shape
    model = get_model(x_input, y_input)
    
    x_train = np.expand_dims(x_train, axis=3)
    x_test = np.expand_dims(x_test, axis=3)
    
    history = model.fit(x_train, 
                    y_train, 
                    validation_data=(x_test,y_test), 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS,
                    verbose=0)
    
    df_history = pd.DataFrame(history.history)
    df_history.to_csv(f'../metrics/{column}_performance.csv', index=False) 

In [134]:
df_metrics = pd.DataFrame(columns=['loss','val_loss', 'root_mean_squared_error','val_root_mean_squared_error'])

files = os.listdir('../metrics')
for file in files:
    input_name = file
    
    path = os.path.join('../metrics', file)
    df_metric = pd.read_csv(path)
    
    df_metrics = df_metrics.append(df_metric.iloc[-1])

files = [f.split('.csv')[0] for f in files]
df_metrics.index = files
df_metrics.sort_values(by=['val_root_mean_squared_error'])


,loss,val_loss,root_mean_squared_error,val_root_mean_squared_error
macd_performance,1365.581598,101.277877,36.953777,10.063691
macd_signal_performance,1283.728022,125.513480,35.829147,11.203279
macd_hist_performance,1304.552247,265.231765,36.118587,16.285936
middle_bband_performance,1568.428488,402.534807,39.603390,20.063271
lower_bband_performance,1401.959678,426.229500,37.442749,20.645327
rsi_performance,1289.459514,484.338536,35.909046,22.007692
k_performance,1146.817463,580.410933,33.864693,24.091719
ema_12_performance,1699.991889,717.158242,41.230957,26.779810
ema_26_performance,868.935446,1155.913964,29.477711,33.998734
upper_bband_performance,1309.532080,1209.742307,36.187458,34.781349


## Prueba utilizando todos los indicadores como inputs

In [137]:
df_min = df[columns]

x, y = utils.create_windowed_dataset(df_min, df['close'].to_numpy(), WINDOW)
x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)

_, x_input, y_input = x.shape
model = get_model(x_input, y_input)

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

history = model.fit(x_train, 
                y_train, 
                validation_data=(x_test,y_test), 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS)

Train on 376 samples, validate on 94 samples
Epoch 1/100
376/376 [==============================] - 1s 4ms/sample - loss: 9849.5600 - root_mean_squared_error: 99.2449 - val_loss: 3301.7088 - val_root_mean_squared_error: 57.4605
Epoch 2/100
376/376 [==============================] - 0s 306us/sample - loss: 3285.9522 - root_mean_squared_error: 57.3232 - val_loss: 277.3692 - val_root_mean_squared_error: 16.6544
Epoch 3/100
376/376 [==============================] - 0s 312us/sample - loss: 2408.1433 - root_mean_squared_error: 49.0728 - val_loss: 477.9353 - val_root_mean_squared_error: 21.8617
Epoch 4/100
376/376 [==============================] - 0s 284us/sample - loss: 2159.5041 - root_mean_squared_error: 46.4705 - val_loss: 336.2963 - val_root_mean_squared_error: 18.3384
Epoch 5/100
376/376 [==============================] - 0s 281us/sample - loss: 2080.9935 - root_mean_squared_error: 45.6179 - val_loss: 269.1131 - val_root_mean_squared_error: 16.4047
Epoch 6/100
376/376 [===============

376/376 [==============================] - 0s 305us/sample - loss: 1205.5102 - root_mean_squared_error: 34.7205 - val_loss: 1229.3313 - val_root_mean_squared_error: 35.0618
Epoch 46/100
376/376 [==============================] - 0s 293us/sample - loss: 1170.3552 - root_mean_squared_error: 34.2105 - val_loss: 641.4318 - val_root_mean_squared_error: 25.3265
Epoch 47/100
376/376 [==============================] - 0s 291us/sample - loss: 1197.3432 - root_mean_squared_error: 34.6026 - val_loss: 615.9873 - val_root_mean_squared_error: 24.8191
Epoch 48/100
376/376 [==============================] - 0s 286us/sample - loss: 1124.7507 - root_mean_squared_error: 33.5373 - val_loss: 664.6319 - val_root_mean_squared_error: 25.7805
Epoch 49/100
376/376 [==============================] - 0s 286us/sample - loss: 1053.8281 - root_mean_squared_error: 32.4627 - val_loss: 889.2665 - val_root_mean_squared_error: 29.8206
Epoch 50/100
376/376 [==============================] - 0s 277us/sample - loss: 1034.75

376/376 [==============================] - 0s 282us/sample - loss: 913.5636 - root_mean_squared_error: 30.2252 - val_loss: 737.8886 - val_root_mean_squared_error: 27.1641
Epoch 90/100
376/376 [==============================] - 0s 284us/sample - loss: 981.8101 - root_mean_squared_error: 31.3338 - val_loss: 988.8694 - val_root_mean_squared_error: 31.4463
Epoch 91/100
376/376 [==============================] - 0s 291us/sample - loss: 1051.1506 - root_mean_squared_error: 32.4215 - val_loss: 780.4427 - val_root_mean_squared_error: 27.9364
Epoch 92/100
376/376 [==============================] - 0s 269us/sample - loss: 1020.9045 - root_mean_squared_error: 31.9516 - val_loss: 951.8690 - val_root_mean_squared_error: 30.8524
Epoch 93/100
376/376 [==============================] - 0s 272us/sample - loss: 1017.0253 - root_mean_squared_error: 31.8908 - val_loss: 1083.3455 - val_root_mean_squared_error: 32.9142
Epoch 94/100
376/376 [==============================] - 0s 269us/sample - loss: 1075.1779